### importing libraries

In [1]:
import numpy as np
import pandas as pd
import random
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Input
from keras import layers
from keras.optimizers import SGD, Adam,RMSprop
from keras.layers import Dense, Dropout, Activation, Flatten,Lambda,Conv1D,  MaxPooling1D
import keras.backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.regularizers import l2, l1
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')
import seaborn as sns
from scipy import stats
from keras.utils.vis_utils import plot_model
import datetime
import umap
import scipy.sparse
import sympy
import sklearn.feature_extraction.text
from sklearn.metrics import classification_report 
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import itertools 
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.SubsMat import MatrixInfo as matlist
from keras.models import load_model
import scipy as sp
from sklearn.neighbors import NearestNeighbors

Using TensorFlow backend.


### function for preprocessing peptides

In [2]:
#adding decoy sequences to the dataset
def add_decoy(data):
    df = pd.concat([data] * 2, ignore_index = True)
    df_len = len(data)
    true_label = [1] * df_len
    print(len(true_label))
    false_label = [0] * df_len
    print(len(false_label))
    labels = np.array(true_label + false_label)
    df['label'] = labels
    df.loc[df_len:, "sequence"] = np.random.permutation(df.loc[df_len:, "sequence"])
    df = df.sample(frac=1.0).reset_index(drop=True)
    return df
def encoding_seqs_left(sequences):
    left = list()
    for i in range(sequences.shape[0]):
        matrix = tokenizer.texts_to_matrix(sequences[i][1]).astype(datatype)[:, 1:]
        matrix = (np.concatenate((matrix, np.zeros((read_length - len(sequences[i][1]), 20)))))
        left.append(matrix)
    sequences_left_matrix=np.array(left)
    return sequences_left_matrix
def encoding_seqs_right(sequences):
    right = list()
    for i in range(len(sequences["predicted_sequence"])):
        matrix = tokenizer.texts_to_matrix(sequences["predicted_sequence"][i]).astype(datatype)[:, 1:]
        matrix = (np.concatenate((matrix, np.zeros((read_length - len(sequences["predicted_sequence"][i]), 20)))))
        right.append(matrix)
    sequences_right_matrix=np.array(right)
    return sequences_right_matrix

### import data -train,validation and test

In [3]:
train = pd.read_csv("./results_train_mod_mass.csv")
val = pd.read_csv("./results_valid_mod_mass.csv")
test = pd.read_csv("./results_test_mod_mass.csv")
print(len(train), len(val),(len(test)))

2076658 259193 259744


### adding decoy peptides to data 

In [1]:
data_train = add_decoy(train)
data_val = add_decoy(val)
# get the subset of the dataset
d_train = data_train.sample(frac=1.0).reset_index(drop=True)
d_val = data_val.sample(frac=1.0).reset_index(drop=True)
# extracting original sequence and its id from training data
X_train_left = d_train.iloc[:, 1:3].values
# extracting predicted deepnovo sequence and its id from training data
X_train_right = pd.concat([d_train.iloc[:,1:2], d_train.iloc[:,3:4]], axis=1)
# extracting labels for the training data
y_train = d_train.iloc[:, 10].values
# extracting original sequence and its id from vaidation data
X_val_left = d_val.iloc[:, 1:3].values
# extracting predicted deepnovo sequence and its id from vaidation data
X_val_right = pd.concat([d_val.iloc[:,1:2], d_val.iloc[:,3:4]], axis=1)
# extracting labels for the training data
y_val = d_val.iloc[:, 10].values 
print(y_train, y_val)

### one-hot encoding of the peptides using kears tokeniser

In [2]:
start = datetime.datetime.now()
alphabet = "ADEFGHKMNPQRSTVWYCLI"
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(alphabet)
datatype = 'int8'
read_length = 40
X_train_left_matrix = encoding_seqs_left(X_train_left)
X_val_left_matrix = encoding_seqs_left(X_val_left)
X_train_right_matrix = encoding_seqs_right(X_train_right)
X_val_right_matrix = encoding_seqs_right(X_val_right)# get the subset of the dataset
print(X_train_left_matrix.shape,X_train_right_matrix.shape,X_val_left_matrix.shape,X_val_right_matrix.shape )
end = datetime.datetime.now()

### functions for model building

In [28]:
#defining distance 
def distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

#defining contrastive loss (taken from keras examples based on Hadsell-et-al.'06)
def contrastive_loss(y_true, y_pred):
    margin = 0.9
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

# Compute classification accuracy.
def accuracy(y_true, y_pred):
    
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

# getting labels from the pedictions probabilities
def predict_class(pred_scores):
    results = {}
    results['predictions_test'] =[]
    results['pred_label'] =[]
    for value in range(len(pred_scores)):
        if pred_scores[value] < 0.5:
            results['predictions_test'].append(pred_scores[value])
            results['pred_label'].append(1)
        else:
            results['predictions_test'].append(pred_scores[value])
            results['pred_label'].append(0)
    return results

In [4]:
#model architecture
input_shape = X_train_right_matrix.shape[1:]
input_shape = X_train_right_matrix.shape[1:]
main_input = Input(shape=input_shape)
x = Conv1D(filters = 40, kernel_size = 3, name = 'conv_1')(main_input) 
x = Conv1D(filters = 80, kernel_size = 3,name = 'conv_2')(x) 
x = MaxPooling1D(pool_size = 2,name = 'pool_one')(x) 
x = Conv1D(filters = 100, kernel_size = 3,kernel_initializer = 'he_uniform', activation = 'relu',kernel_regularizer=l2(0.01),  name = 'conv_3')(x) 
x = Dropout(0.3)(x) 
x = Flatten(name = 'flatten')(x) 
x= Dropout(0.3)(x)
x = Dense(128, kernel_initializer= 'he_uniform', activation = 'relu', kernel_regularizer=l2(0.01), name = 'dense_one')(x)
x = Dense(64, kernel_initializer= 'he_uniform', activation = 'relu', kernel_regularizer=l2(0.01), name = 'dense_two')(x)
network = Model(main_input, x)
# network definition
left_input = Input(shape=input_shape) #encoding for standard 20AAs, start token, stop token, Nmod,Mmod,Qmod and '/'
right_input = Input(shape=input_shape)

In [3]:
'''
Because we re-use the same instance `network`,
the weights of the network
will be shared across the two branches
'''
feat_map_l = network(left_input)
feat_map_r = network(right_input)
print(network.summary())
L1_layer = Lambda(distance, output_shape=dist_output_shape)
L1_distance = L1_layer([feat_map_l, feat_map_r])
siamese_model = Model(inputs=[left_input, right_input],outputs=L1_distance)
siamese_model.summary()
epochs=20
lrate=0.001

# compiling the model
siamese_model.compile(loss=contrastive_loss,optimizer= Adam(lrate, decay=1.5e-4),metrics=[accuracy] )
callbacks = [
    EarlyStopping(monitor='val_accuracy', mode ='max', verbose=1),
    ModelCheckpoint('best_model.h5', monitor='val_accuracy',mode='max',verbose=1)]
history = siamese_model.fit([X_train_left_matrix, X_train_right_matrix],y_train,epochs=epochs, batch_size = 32, validation_data=([X_val_left_matrix, X_val_right_matrix], y_val), callbacks=callbacks)
end1 = datetime.datetime.now()
siamese_model.save_weights('./model.h5')
plot_model(siamese_model, to_file= './final_model.h5')

### visualising results

In [ ]:
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Training loss vs Validation loss')
plt.legend()
plt.savefig("./loss.png")
plt.show()

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.title('Training accuracy vs Validation accuracy')
plt.legend(['train', 'test'], loc='lower right')
plt.savefig("./accuracy.png")
plt.show()

# # #  *validation model against unknown datasets*

In [5]:
# data preparation for testing the accuracy of the model
def check_model(data):
    test_data = add_decoy(data)
    X_sample_left = test_data.iloc[:, 1:3].values
    X_sample_right = pd.concat([test_data.iloc[:,1:2], test_data.iloc[:,3:4]], axis=1)
    y_sample = test_data.iloc[:, 10].values
    X_sample_left_matrix = encoding_seqs_left(X_sample_left)
    X_sample_right_matrix = encoding_seqs_right(X_sample_right)
    model_pred = loaded_Smodel.predict([X_sample_left_matrix, X_sample_right_matrix], batch_size = 32, verbose=0)
    accuracy = loaded_Smodel.evaluate([X_sample_left_matrix, X_sample_right_matrix], y_sample, batch_size=128)
    print(accuracy)
    test_data['model_prediction'] = model_pred
    return test_data, y_sample, model_pred
human_test_data , y_test, human_pred = check_model(test)    

In [6]:
yeast_data = pd.read_csv("./results_yeast_mod_mass_new.csv")
yeast_test_data, y_yeast, yeast_pred = check_model(yeast_data)

In [7]:
mouse_data = pd.read_csv("./test_mouse_10k.csv")
mouse_test_data, y_mouse, mouse_pred = check_model(mouse_data)

In [8]:
# performance metric for the model when testes on human test data
human_against_human_df = pd.DataFrame.from_dict(predict_class(human_pred))
print(human_against_human_df.head()) 

hh_probs = human_against_human_df['predictions_test']

print(type(human_against_human_df['pred_label']))

y_pred_human =pd.Series(human_against_human_df['pred_label']).values

print(type(y_test), type(y_pred_human))

# y_test = y_test and y_pred = y_pred_human
print(classification_report(y_test, y_pred_human))

# get the auc score
auc_human = roc_auc_score(y_test, y_pred_human)
print(auc_human)

cm = confusion_matrix(y_test, y_pred_human)
cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title('confusion matrix for the human test data')
plt.colorbar()
plt.xlabel('True label')
plt.ylabel('Predicted label')
plt.xticks([0, 1])
plt.yticks([0, 1])
plt.grid('off')
plt.savefig('./confusion_matrix_human')
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], '.2f'),
             horizontalalignment='center',
             color='white' if cm[i, j] > 0.5 else 'black')


In [9]:
# ROC curve kears  taken from(https://github.com/Tony607/ROC-Keras/blob/master/ROC-Keras.ipynb)
from sklearn.metrics import roc_curve
fpr_human, tpr_human, thresholds_human = roc_curve(y_test, y_pred_human)
fpr_yeast, tpr_yeast, thresholds_yeast = roc_curve(y_yeast, y_pred_yeast)
fpr_mouse, tpr_mouse, thresholds_mouse = roc_curve(y_mouse, y_pred_mouse)
from sklearn.metrics import auc
auc_human = auc(fpr_human, tpr_human)
auc_yeast = auc(fpr_yeast, tpr_yeast)
auc_mouse = auc(fpr_mouse, tpr_mouse)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_human, tpr_human, label='human (area = {:.3f})'.format(auc_human))
plt.plot(fpr_yeast, tpr_yeast, label='yeast (area = {:.3f})'.format(auc_yeast))
plt.plot(fpr_mouse, tpr_mouse, label='mouse (area = {:.3f})'.format(auc_mouse))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.savefig('./ROC_curve')
plt.show()

### function to calculate the peptide mass

In [35]:
aminoacid = {
    'A': 71.03711, # 0
           'R': 156.10111, # 1
           'N': 114.04293, # 2
           'D': 115.02694, # 3
           'C': 160.03065, # 4
           'E': 129.04259, # 5
           'Q': 128.05858, # 6
           'G': 57.02146, # 7
           'H': 137.05891, # 8
           'I': 113.08406, # 9
           'L': 113.08406, # 10
           'K': 128.09496, # 11
           'M': 131.04049, # 12
           'F': 147.06841, # 13
           'P': 97.05276, # 14
           'S': 87.03203, # 15
           'T': 101.04768, # 16
           'W': 186.07931, # 17
           'Y': 163.06333, # 18
           'V': 99.06841, # 19
}

N_terminus_mass = 1.00728
C_terminus_mass = 19.0178

def protein_mass(protein):
    temp =0
    for char in protein:
        temp += aminoacid[char]
    tot_mol_wt =N_terminus_mass + temp + C_terminus_mass 
    return tot_mol_wt

protein_mass('DIGAIVYGFPNK')

1294.69097

### extracting embedding vector from the model

In [33]:
def extract_embedding(sample):
    X_trial_right = pd.concat([sample.iloc[:,1:2], sample.iloc[:,3:4]], axis=1)
    X_trial_right_matrix = encoding_seqs_right(X_trial_right)
    embedding_query_sample_subset = intermediate_layer_model.predict(X_trial_right_matrix)
    return embedding_query_sample_subset

In [10]:
from keras.models import load_model
import scipy as sp

# In functional api when you call the model you are reusing the architecture and its weights automatically.
intermediate_layer_model = Model(inputs=[network.layers[0].input],outputs=network.get_layer('dense_two').output)
intermediate_layer_model.summary()
# dropping duplicates from the training dataset and extracting embedding vector 
train_hu = pd.read_csv("./results_train_mod_mass.csv")
train_unique = (pd.DataFrame(train_hu, columns=['scan','sequence'])).drop_duplicates('sequence')
print(len(train_unique))
X_left_tr = train_unique.iloc[:, :].values
X_h_left_matrix = encoding_seqs_left(X_left_tr)
embedding_ref_hu_subset = intermediate_layer_model.predict(X_h_left_matrix)
# getting a subset for trial
human_trial = data_train.iloc[:101,:]
embedding_query_human_subset = extract_embedding(human_trial)

In [36]:
# embedding vector for mouse sample data consisting of 101 peptides
mouse_data = pd.read_csv("./test_mouse_10k.csv")
mouse_trial = mouse_data.iloc[:101,:]
embedding_query_mouse_subset = extract_embedding(mouse_trial)

In [11]:
# embedding vector for yeast sample data consisting of 101 peptides
yeast_trial = yeast_data.iloc[:101,:]
embedding_query_yeast_subset = extract_embedding(yeast_trial)
print(len(embedding_ref_hu_subset), len(embedding_query_yeast_subset))

### function for NearestNeighbors for unlabelled data

In [38]:
from sklearn.neighbors import NearestNeighbors
# calculating NNs for query sequence
nbrs = NearestNeighbors(n_neighbors=100, algorithm='auto').fit(embedding_ref_hu_subset)
def nearest_neighbor(sample_dataset, ref_dataset, embedding_query_subset):
    query_index = sample_dataset.iloc[:,3:4].shape[0]
    query_sequence = sample_dataset.iloc[:,3:4].reset_index(drop=True)
    pepmass = (sample_dataset.iloc[:,-2].reset_index(drop=True)) * (sample_dataset.iloc[:,-1].reset_index(drop=True))
    query_nn = pd.DataFrame(columns=['index','scan','sequence','predicted_sequence', 'dist', 'query_peptide','ref_peptide', 'mass_diff'])
    KNNs=[]
    for i in range(0,query_index):
        q_seq = query_sequence['predicted_sequence'][i]
        query_peptide = pepmass[i]
        distances, indices = nbrs.kneighbors(embedding_query_subset[i:i+1,:])
        temp = set()
        NNs=[]
        for j in range(0,distances.size):
            seq_row = indices.flatten()[j]
            scan = ref_dataset.iloc[seq_row,1:2][0]
            ref_sequence = ref_dataset.iloc[seq_row,2:3][0]
            ref_peptide = protein_mass(ref_sequence)
            mass_diff = abs(ref_peptide-query_peptide)          
            dist = distances.flatten()[j] 
            if not ref_sequence in temp:
                NNs.append( (seq_row,scan,ref_sequence, q_seq, dist,query_peptide, ref_peptide, mass_diff) )
                NNs.sort(key=lambda tup: tup[7])
                temp.add(ref_sequence)
            else:
                continue
        neighbors = NNs[:]
        for element in neighbors:
            KNNs.append( (element[0], element[1], element[2], element[3], element[4], element[5], element[6], element[7]) )
    f = lambda x,index:tuple( i[index] for i in x)
    tup1 = f(KNNs,0)
    tup2 = f(KNNs,1)
    tup3 = f(KNNs,2)
    tup4 = f(KNNs,3)
    tup5 = f(KNNs,4)
    tup6 = f(KNNs,5)
    tup7 = f(KNNs,6)
    tup8 = f(KNNs,7)
    for x in range(len(KNNs)):
        query_nn = query_nn.append({'index':tup1[x], 'scan': tup2[x], 'sequence':tup3[x],'predicted_sequence':tup4[x],'dist':tup5[x], 'query_peptide':tup6[x], 'ref_peptide':tup7[x], 'mass_diff':tup8[x] },ignore_index=True)
    return query_nn

In [12]:
yeast_nn = nearest_neighbor(yeast_trial, data_train,embedding_query_yeast_subset)
mouse_nn = nearest_neighbor(mouse_trial, data_train,embedding_query_mouse_subset)
yeast_nn.to_csv('./yeast_nn.csv', index=False)
mouse_nn.to_csv('./mouse_nn.csv', index=False)

### Evaluating NearestNeibors results with Blosum score

In [40]:
def blosum_score(sample):
    validation = pd.DataFrame(columns=['predicted_sequence','scan','ref_sequence','nn_dist','blosum_score'])
    scores = []
    for i in range(0,len(sample)):
        query_sequence = sample['predicted_sequence'][i]
        ref_id = sample['scan'][i]
        ref_sequence = sample['sequence'][i]
#         label_gt = sample['label'][i]
        nn_dist = sample['dist'][i]
        x,y = (query_sequence,ref_sequence)
        score = pairwise2.align.globalds(x, y, matrix, -9,-1)
        score_norm = score[0][2] / score[0][4]
        scores.append( ( query_sequence,ref_id, ref_sequence, nn_dist,score_norm) )
    f = lambda x,index:tuple( i[index] for i in x)
    tup1 = f(scores,0)
    tup2 = f(scores,1)
    tup3 = f(scores,2)
    tup4 = f(scores,3)
    tup5= f(scores,4)
    for x in range(len(scores)):
        validation = validation.append({ 'predicted_sequence':tup1[x],'scan': tup2[x],'ref_sequence':tup3[x],'nn_dist':tup4[x],'blosum_score':tup5[x]},ignore_index=True)
    return validation

In [13]:
X_left = yeast_nn.iloc[:, 1:3].values
X_right = pd.concat([yeast_nn.iloc[:,1:2], yeast_nn.iloc[:,3:4]], axis=1)
X_left_matrix = encoding_seqs_left(X_left)
X_right_matrix = encoding_seqs_right(X_right)
print(X_left_matrix.shape,X_right_matrix.shape)
yeast_nn_blosum = blosum_score(yeast_nn)
yeast_nn_blosum.to_csv('./yeast_nn_blosum.csv', index=False)

In [14]:
X_left = mouse_nn.iloc[:, 1:3].values
X_right = pd.concat([mouse_nn.iloc[:,1:2], mouse_nn.iloc[:,3:4]], axis=1)
X_left_matrix = encoding_seqs_left(X_left)
X_right_matrix = encoding_seqs_right(X_right)
print(X_left_matrix.shape,X_right_matrix.shape)
mouse_nn_blosum = blosum_score(mouse_nn)
mouse_nn_blosum.to_csv('./mouse_nn_blosum.csv', index=False)

### Parsing human proteome from Uniprot and preprocessing data for database search

In [15]:
from Bio import SeqIO
import gzip
import os
from pyteomics import fasta, parser, mass, achrom, electrochem, auxiliary
records = list(SeqIO.parse("./Human.fasta", "fasta"))
with open("./Human.fasta", "r") as handle:
    record_iterator = SeqIO.parse(handle, "fasta")
    record_list = list(record_iterator)
    sequence_list = [str(record.seq) for record in record_list]
    print("Number of protein sequences: {0:d}".format(len(sequence_list)))

In [ ]:
unique_peptides = set()
for sequence in sequence_list:
    unique_peptides.update((parser.cleave(sequence=sequence,
          rule=parser.expasy_rules['trypsin'],
          missed_cleavages=2)))
    peptide_list = list(unique_peptides)
peptide_count = len(peptide_list)
print("Number of peptides: {0:d}".format(peptide_count))

In [ ]:
# skip peptides with undetermined amino acid 'X', or 'B'
peptide_list = [list(peptide) for peptide in peptide_list
                if not any(x in peptide for x in ['X', 'B', 'U', 'Z'])]
peptide_count = len(peptide_list)
peptide_list = ["".join(x) for x in peptide_list]
print("Number of peptides: {0:d}".format(peptide_count))

In [ ]:
# for each peptide, find the mass
peptide_mass_array = np.zeros(peptide_count)
for index, peptide in enumerate(peptide_list):
    peptide_mass_array[index] = protein_mass(peptide)
db_data = {peptide_list[i]: peptide_mass_array[i] for i in range(len(peptide_list))} 
type(db_data)
df_data = pd.DataFrame(db_data.items(), columns=['sequence','pep_mass'])
df_data['sequence'] = df_data['sequence'].str.replace(r'[IL]','I')
df_data = df_data.drop_duplicates('sequence')
print(len(df_data))
df_data.head()

### function to perform database search and calculate homologues based on precursor mass

In [ ]:
precursor_mass_ppm = 20.0/1000000
def database_search(query_database, ref_database):
    df = pd.DataFrame(columns=['query_id','query_seq','ref_sequence','ref_mass', 'query_mass', 'mass_diff'])
    hits =[]
    for i in range(0,len(query_database)):
        query_id = query_database['scan'][i]
        query_sequence = query_database['predicted_sequence'][i]
        experimental_mass = query_database['mass'][i] 
        sequence_mass = protein_mass(query_sequence) 
        precursor_mass_tolerance = experimental_mass * precursor_mass_ppm
        for j in range(0,len(ref_database)):
            ref_sequence = ref_database['sequence'][j]
            pepmass = ref_database['pep_mass'][j]
            if abs(float(pepmass) - sequence_mass) <= precursor_mass_tolerance:
                mass_diff = abs(float(pepmass) - sequence_mass)
                hits.append( ( query_id,query_sequence, ref_sequence,pepmass, sequence_mass,mass_diff ) )
    f = lambda x,index:tuple( i[index] for i in x)
    tup1 = f(hits,0)
    tup2 = f(hits,1)
    tup3 = f(hits,2)
    tup4 = f(hits,3)
    tup5 = f(hits,4)
    tup6 = f(hits,5)
    for i in range(len(hits)):
        df = df.append({'query_id': tup1[i], 'query_seq':tup2[i],'ref_sequence':tup3[i],'ref_mass':tup4[i], 'query_mass':tup5[i], 'mass_diff': tup6[i]},ignore_index=True)
    return df

In [ ]:
yeast_data = pd.read_csv("./results_yeast_mod_mass_new.csv")
yeast_trial = yeast_data.iloc[:101,:]
pepmass = (yeast_trial.iloc[:,-2].reset_index(drop=True)) * (yeast_trial.iloc[:,-1].reset_index(drop=True))
yeast_trial['mass'] = pepmass
yeast_trial.tail()
yeast_hits = database_search(yeast_trial, df_data)
yeast_hits.to_csv('./yeast_hits.csv', index=False)

In [ ]:
mouse_data = pd.read_csv("./test_mouse_10k.csv")
mouse_trial = mouse_data.iloc[:101,:]
pepmass = (mouse_trial.iloc[:,-2].reset_index(drop=True)) * (mouse_trial.iloc[:,-1].reset_index(drop=True))
mouse_trial['mass'] = pepmass
mouse_trial.tail()
mouse_hits = database_search(mouse_trial, df_data)
mouse_hits.to_csv("./mouse_hits.csv", index=False)

### Evaluate database search results with blosum score to minimise false positives

In [ ]:
from Bio import pairwise2
# Import format_alignment method
from Bio.pairwise2 import format_alignment
from Bio.SubsMat import MatrixInfo as matlist
matrix = matlist.blosum62
def blosum_score(sample):
    validation = pd.DataFrame(columns=['predicted_sequence','scan','ref_sequence','mass_diff','blosum_score'])
    scores = []
    for i in range(0,len(sample)):
        query_sequence = sample['query_seq'][i]
        ref_id = sample['query_id'][i]
        ref_sequence = sample['ref_sequence'][i]
#         label_gt = sample['label'][i]
        mass_diff = sample['mass_diff'][i]
        x,y = (query_sequence,ref_sequence)
        score = pairwise2.align.globalds(x, y, matrix, -9,-1)
        score_norm = score[0][2] / score[0][4]
        scores.append( ( query_sequence,ref_id, ref_sequence, mass_diff,score_norm) )
    f = lambda x,index:tuple( i[index] for i in x)
    tup1 = f(scores,0)
    tup2 = f(scores,1)
    tup3 = f(scores,2)
    tup4 = f(scores,3)
    tup5= f(scores,4)
    for x in range(len(scores)):
        validation = validation.append({ 'predicted_sequence':tup1[x],'scan': tup2[x],'ref_sequence':tup3[x],'mass_diff':tup4[x],'blosum_score':tup5[x]},ignore_index=True)
    return validation

In [ ]:
yeast_db_blosum = blosum_score(yeast_hits)
yeast_db_blosum.to_csv('./yeast_db_blosum.csv', index=False)
mouse_db_blosum = blosum_score(mouse_hits)
mouse_db_blosum.to_csv('./mouse_db_blosum.csv', index=False)

### metrics for model performance in predicting homologues using cross-species proteome

In [16]:
mouse_db = pd.read_csv("./mouse_db_blosum.csv")
mouse_db_filtered= mouse_db.loc[mouse_db["blosum_score"] >=1]
mouse_hit_count = mouse_db_filtered.groupby(["predicted_sequence"]).size().reset_index(name='database hits')
mouse_hit_count

In [17]:
mouse_nn=pd.read_csv("./mouse_nn_blosum.csv")
mouse_nn_filtered=mouse_nn.loc[mouse_nn["blosum_score"]>=1]
mouse_hits= mouse_nn_filtered.groupby(["predicted_sequence"]).size().reset_index(name='KNN hits')
print(mouse_hits)
merged_inner_mouse = pd.merge(left=mouse_hit_count, right=mouse_hits,  left_on='predicted_sequence', right_on='predicted_sequence')
merged_mouse = merged_inner_mouse.sort_values("database hits", ascending=False)
print(merged_mouse)
merged_mouse.plot(x="predicted_sequence",y=["database hits", "KNN hits"],  kind="barh")


In [18]:
yeast_db = pd.read_csv("./yeast_db_blosum.csv")
yeast_db_filtered= yeast_db.loc[yeast_db["blosum_score"] >=1]
yeast_hit_count = yeast_db_filtered.groupby(["predicted_sequence"]).size().reset_index(name='database hits')
print(yeast_hit_count)
yeast_nn=pd.read_csv("./yeast_nn_blosum.csv")
yeast_nn_filtered=yeast_nn.loc[yeast_nn["blosum_score"] >=1]
yeast_hits = yeast_nn_filtered.groupby(["predicted_sequence"]).size().reset_index(name='KNN hits')
print(yeast_hits)
merged_inner_yeast = pd.merge(left=yeast_hit_count, right=yeast_hits, left_on='predicted_sequence', right_on='predicted_sequence')
merged_yeast = merged_inner_yeast.sort_values("database hits", ascending=False)
print(merged_yeast)
merged_yeast.plot(x="predicted_sequence",y=["database hits", "KNN hits"],  kind="barh")